# Installation

In [ ]:
!pip install git+https://github.com/suno-ai/bark.git
!git clone https://github.com/suno-ai/bark.git
!git clone https://github.com/gitmylo/bark-voice-cloning-HuBERT-quantizer
!pip install -r ./bark-voice-cloning-HuBERT-quantizer/requirements.txt
!mv bark-voice-cloning-HuBERT-quantizer bark_voice_cloning_HuBERT_quantizer

  Cloning https://github.com/suno-ai/bark.git to /tmp/pip-req-build-8xz_qler
  Running command git clone --filter=blob:none --quiet https://github.com/suno-ai/bark.git /tmp/pip-req-build-8xz_qler
  Resolved https://github.com/suno-ai/bark.git to commit 773624d26db84278a55aacae9a16d7b25fbccab8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 24.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 64.6 MB/s eta 0:00:00
   

In [ ]:
!pip list | grep sox

sox                              1.5.0
soxr                             0.3.7


In [ ]:
import sys
sys.path.append('./bark_voice_cloning_HuBERT_quantizer')
import os
from scipy.io.wavfile import write as write_wav
import numpy as np
import torch
import torchaudio
from bark.api import generate_audio
from bark.generation import SAMPLE_RATE, preload_models, load_codec_model
from encodec.utils import convert_audio
from bark_voice_cloning_HuBERT_quantizer.bark_hubert_quantizer.customtokenizer import CustomTokenizer
from bark_voice_cloning_HuBERT_quantizer.bark_hubert_quantizer.hubert_manager import HuBERTManager
from bark_voice_cloning_HuBERT_quantizer.bark_hubert_quantizer.pre_kmeans_hubert import CustomHubert
from bark.api import semantic_to_waveform
from bark import generate_audio, SAMPLE_RATE

In [ ]:
torch.cuda.is_available()

True

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
use_gpu = True if device == 'cuda' else False
model = load_codec_model(use_gpu=use_gpu)

preload_models(
    text_use_gpu=use_gpu,
    text_use_small=False,
    coarse_use_gpu=use_gpu,
    coarse_use_small=False,
    fine_use_gpu=use_gpu,
    fine_use_small=False,
    codec_use_gpu=use_gpu,
    force_reload=False
)

hubert_manager = HuBERTManager()
#hubert_manager.make_sure_hubert_installed()
#hubert_manager.make_sure_tokenizer_installed()

# Load the HuBERT model
hubert_model = CustomHubert(checkpoint_path='data/models/hubert/hubert.pt').to(device)

# Load the CustomTokenizer model
tokenizer = CustomTokenizer.load_from_checkpoint('data/models/hubert/tokenizer.pth', map_location=device).to(device)

/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


# Inference

In [ ]:
text_prompt = "Imagine a world where technology and creativity dance together, creating a symphony of innovation. [laughs] In this realm, ideas come to life, propelled by the power of artificial intelligence. It's a place where every line of code is a brushstroke on the canvas of possibility." #@param {type:"string"}
audio_filepath = '/content/Recording.wav' #@param {type:"string"}

if not os.path.isfile(audio_filepath):
  raise ValueError(f"Audio file not exists ({audio_filepath})")

wav, sr = torchaudio.load(audio_filepath)
wav = convert_audio(wav, sr, model.sample_rate, model.channels)
wav = wav.to(device)

semantic_vectors = hubert_model.forward(wav, input_sample_hz=model.sample_rate)
semantic_tokens = tokenizer.get_token(semantic_vectors)

# Extract discrete codes from EnCodec
with torch.no_grad():
    encoded_frames = model.encode(wav.unsqueeze(0))
codes = torch.cat([encoded[0] for encoded in encoded_frames], dim=-1).squeeze()

# move codes to cpu
codes = codes.cpu().numpy()
# move semantic tokens to cpu
semantic_tokens = semantic_tokens.cpu().numpy()

voice_filename = 'output.npz'

current_path = os.getcwd()
voice_name = os.path.join(current_path, voice_filename)

np.savez(voice_name, fine_prompt=codes, coarse_prompt=codes[:2, :], semantic_prompt=semantic_tokens)

# simple generation
audio_array = generate_audio(text_prompt, history_prompt=voice_name, text_temp=0.7, waveform_temp=0.7)

# save audio
filepath = "out.wav" # change this to your desired output path
write_wav(filepath, SAMPLE_RATE, audio_array)

100%|██████████| 31/31 [00:30<00:00,  1.03it/s]


In [ ]:
def voice_cloner(audio_file, text_prompt):
  device = 'cuda' if torch.cuda.is_available() else 'cpu'
  use_gpu = True if device == 'cuda' else False
  model = load_codec_model(use_gpu=use_gpu)

  preload_models(
      text_use_gpu=use_gpu,
      text_use_small=False,
      coarse_use_gpu=use_gpu,
      coarse_use_small=False,
      fine_use_gpu=use_gpu,
      fine_use_small=False,
      codec_use_gpu=use_gpu,
      force_reload=False
  )

  hubert_manager = HuBERTManager()
  hubert_manager.make_sure_hubert_installed()
  hubert_manager.make_sure_tokenizer_installed()

  # Load the HuBERT model
  hubert_model = CustomHubert(checkpoint_path='data/models/hubert/hubert.pt').to(device) # should use large resources

  # Load the CustomTokenizer model
  tokenizer = CustomTokenizer.load_from_checkpoint('data/models/hubert/tokenizer.pth', map_location=device).to(device) # should use large resources
  ##### I SHOULD LOAD VOICE CORRECTLY
  wav, sr = torchaudio.load(audio_file)
  wav = convert_audio(wav, sr, model.sample_rate, model.channels)
  wav = wav.to(device)

  semantic_vectors = hubert_model.forward(wav, input_sample_hz=model.sample_rate)
  semantic_tokens = tokenizer.get_token(semantic_vectors)

  # Extract discrete codes from EnCodec
  with torch.no_grad():
      encoded_frames = model.encode(wav.unsqueeze(0))
  codes = torch.cat([encoded[0] for encoded in encoded_frames], dim=-1).squeeze()

  # move codes to cpu
  codes = codes.cpu().numpy()
  # move semantic tokens to cpu
  semantic_tokens = semantic_tokens.cpu().numpy()

  voice_filename = 'output.npz'

  current_path = os.getcwd()
  voice_name = os.path.join(current_path, voice_filename)

  np.savez(voice_name, fine_prompt=codes, coarse_prompt=codes[:2, :], semantic_prompt=semantic_tokens)
  #SEND ``.npz`` file format TO BACKEND


In [ ]:
def voice_generator(npz_file, text_prompt, Gen_temp = 0.6):
  text_prompt = text_prompt.replace("\n", " ").strip()
  # MAY BE I SHOULD PROCESS npz_file some how base on it's nature. URL or ......
  GEN_TEMP = Gen_temp
  sentences = nltk.sent_tokenize(text_prompt)
  silence = np.zeros(int(0.10 * SAMPLE_RATE))  # quarter second of silence

  pieces = []
  for sentence in sentences:
      semantic_tokens = generate_text_semantic(
          sentence,
          history_prompt=npz_file, # SPEAKER, npz file
          temp=GEN_TEMP,
          min_eos_p=0.05,  # this controls how likely the generation is to end
      )

      audio_array = semantic_to_waveform(semantic_tokens, history_prompt=npz_file,)
      pieces += [audio_array, silence.copy()]
  final_voice = np.concatenate(pieces)
  import soundfile as sf

  # Write out audio as 24bit flac, change accordingly
  # send it to backend
  sf.write('audio.flac', final_voice, SAMPLE_RATE, format='flac', subtype='PCM_24')


In [ ]:
from bark.api import generate_audio
from transformers import BertTokenizer
from bark.generation import SAMPLE_RATE, preload_models, codec_decode, generate_coarse, generate_fine, generate_text_semantic

# Enter your prompt and speaker here
#text_prompt = "Hello, my name is Serpy. And, uh — and I like pizza. [laughs]"
#voice_name = "output" # use your custom voice name here if you have one

In [ ]:
# generation with more control
x_semantic = generate_text_semantic(
    text_prompt,
    history_prompt=voice_name,
    temp=0.7,
    top_k=50,
    top_p=0.95,
)

x_coarse_gen = generate_coarse(
    x_semantic,
    history_prompt=voice_name,
    temp=0.7,
    top_k=50,
    top_p=0.95,
)
x_fine_gen = generate_fine(
    x_coarse_gen,
    history_prompt=voice_name,
    temp=0.5,
)
audio_array = codec_decode(x_fine_gen)

100%|██████████| 37/37 [07:26<00:00, 12.06s/it]


In [ ]:
from IPython.display import Audio
# play audio
Audio(audio_array, rate=SAMPLE_RATE)

In [ ]:
from scipy.io.wavfile import write as write_wav
# save audio
filepath = "audio.wav" # change this to your desired output path
write_wav(filepath, SAMPLE_RATE, audio_array)

# Long form text generation

In [ ]:
import nltk  # we'll use this to split into sentences
import numpy as np
nltk.download('punkt')
from bark.api import semantic_to_waveform
from bark import generate_audio, SAMPLE_RATE

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
script = """
If you’re looking to invest in property,you may be wondering if it’s possible to find good deals on off-market properties. While you may have to do some digging to find these listings,they can be a great resource for finding good deals on investment properties. If you’re looking to invest in property, you may be wondering if it’s possible to find good deals on properties that are off-market.
""".replace("\n", " ").strip()

In [ ]:
sentences = nltk.sent_tokenize(script)

# generation

In [ ]:
GEN_TEMP = 0.6
#SPEAKER = "v2/en_speaker_6"
silence = np.zeros(int(0.15 * SAMPLE_RATE))  # quarter second of silence

pieces = []
for sentence in sentences:
    semantic_tokens = generate_text_semantic(
        sentence,
        history_prompt=voice_name, # SPEAKER,
        temp=GEN_TEMP,
        min_eos_p=0.05,  # this controls how likely the generation is to end
    )

    audio_array = semantic_to_waveform(semantic_tokens, history_prompt=voice_name,)
    pieces += [audio_array, silence.copy()]


100%|██████████| 20/20 [00:18<00:00,  1.06it/s]


In [ ]:
audio_array.shape

(189440,)

In [ ]:
np.concatenate(pieces).shape

(609200,)

In [ ]:
from IPython.display import Audio
Audio(np.concatenate(pieces), rate=SAMPLE_RATE)

In [ ]:
from scipy.io.wavfile import write as write_wav
# save audio
filepath = "audio.wav" # change this to your desired output path
write_wav(filepath, SAMPLE_RATE, np.concatenate(pieces))

In [ ]:
import soundfile as sf

# Write out audio as 24bit flac, change accordingly
sf.write('audio.flac', np.concatenate(pieces), SAMPLE_RATE, format='flac', subtype='PCM_24')

# Audio Quality Check

In [ ]:
!git clone https://github.com/gabrielmittag/NISQA.git

fatal: destination path 'NISQA' already exists and is not an empty directory.


In [ ]:
!pip install -r /content/NISQA/env.yml

ERROR: Invalid requirement: 'name: nisqa' (from line 1 of /content/NISQA/env.yml)


In [ ]:
import os
import yaml

with open("/content/NISQA/env.yml") as file_handle:
    environment_data = yaml.safe_load(file_handle)

for dependency in environment_data["dependencies"]:
    if isinstance(dependency, dict):
      for lib in dependency['pip']:
        os.system(f"pip install {lib}")

In [ ]:
!python /content/NISQA/run_predict.py --mode predict_file --pretrained_model weights/nisqa.tar --deg /content/Recording.wav --output_dir /content/results

Traceback (most recent call last):
  File "/content/NISQA/run_predict.py", line 5, in <module>
    from nisqa.NISQA_model import nisqaModel
  File "/content/NISQA/nisqa/NISQA_model.py", line 19, in <module>
    from . import NISQA_lib as NL
  File "/content/NISQA/nisqa/NISQA_lib.py", line 10, in <module>
    import librosa as lb
  File "/usr/local/lib/python3.10/dist-packages/librosa/__init__.py", line 211, in <module>
    from . import core
  File "/usr/local/lib/python3.10/dist-packages/librosa/core/__init__.py", line 9, in <module>
    from .constantq import *  # pylint: disable=wildcard-import
  File "/usr/local/lib/python3.10/dist-packages/librosa/core/constantq.py", line 1059, in <module>
    dtype=np.complex,
  File "/usr/local/lib/python3.10/dist-packages/numpy/__init__.py", line 319, in __getattr__
    raise AttributeError(__former_attrs__[attr])
AttributeError: module 'numpy' has no attribute 'complex'.
`np.complex` was a deprecated alias for the builtin `complex`. To avoid t

In [ ]:
!pip list | grep numpy

numpy                            1.25.2


In [ ]:
!pip uninstall numpy -y
!pip install numpy==1.20.3

Found existing installation: numpy 1.25.2
Uninstalling numpy-1.25.2:
  Successfully uninstalled numpy-1.25.2
  Using cached numpy-1.20.3.zip (7.8 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for numpy (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for numpy
Failed to build numpy
ERROR: Could not build wheels for numpy, which is required to install pyproject.toml-based projects


In [ ]:
!pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2


In [ ]:
import argparse
parser = argparse.ArgumentParser()

In [ ]:
parser

ArgumentParser(prog='colab_kernel_launcher.py', usage=None, description=None, formatter_class=<class 'argparse.HelpFormatter'>, conflict_handler='error', add_help=True)

In [ ]:

parser.add_argument('--mode',  type=str, help='either predict_file, predict_dir, or predict_csv')
parser.add_argument('--pretrained_model', type=str,
                        help='file name of pretrained model (must be in current working folder)')
parser.add_argument('--deg', type=str, help='path to speech file')

_StoreAction(option_strings=['--deg'], dest='deg', nargs=None, const=None, default=None, type=<class 'str'>, choices=None, required=False, help='path to speech file', metavar=None)

In [ ]:
args = parser.parse_args()
args = vars(args)

usage: colab_kernel_launcher.py [-h] [--mode MODE] [--pretrained_model PRETRAINED_MODEL]
                                [--deg DEG]
colab_kernel_launcher.py: error: unrecognized arguments: -f /root/.local/share/jupyter/runtime/kernel-1f7998b1-dd58-4911-880c-8905a3734624.json


SystemExit: 2